**Modules**

In [1]:
import pandas as pd
import json
import gzip
import gc

### Loading Complete Dataset

**Streaming the data without unzipping the complete file into the memory**
- Reading data one line at a time

In [3]:
with gzip.open("../Initial/books.json.gz") as f:
    line = f.readline()

line

b'{"isbn": "0312853122", "text_reviews_count": "1", "series": [], "country_code": "US", "language_code": "", "popular_shelves": [{"count": "3", "name": "to-read"}, {"count": "1", "name": "p"}, {"count": "1", "name": "collection"}, {"count": "1", "name": "w-c-fields"}, {"count": "1", "name": "biography"}], "asin": "", "is_ebook": "false", "average_rating": "4.00", "kindle_asin": "", "similar_books": [], "description": "", "format": "Paperback", "link": "https://www.goodreads.com/book/show/5333265-w-c-fields", "authors": [{"author_id": "604031", "role": ""}], "publisher": "St. Martin\'s Press", "num_pages": "256", "publication_day": "1", "isbn13": "9780312853129", "publication_month": "9", "edition_information": "", "publication_year": "1984", "url": "https://www.goodreads.com/book/show/5333265-w-c-fields", "image_url": "https://images.gr-assets.com/books/1310220028m/5333265.jpg", "book_id": "5333265", "ratings_count": "3", "work_id": "5400751", "title": "W.C. Fields: A Life on Film", "t

**json to dict**

In [4]:
line = json.loads(line)
line

{'isbn': '0312853122',
 'text_reviews_count': '1',
 'series': [],
 'country_code': 'US',
 'language_code': '',
 'popular_shelves': [{'count': '3', 'name': 'to-read'},
  {'count': '1', 'name': 'p'},
  {'count': '1', 'name': 'collection'},
  {'count': '1', 'name': 'w-c-fields'},
  {'count': '1', 'name': 'biography'}],
 'asin': '',
 'is_ebook': 'false',
 'average_rating': '4.00',
 'kindle_asin': '',
 'similar_books': [],
 'description': '',
 'format': 'Paperback',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'authors': [{'author_id': '604031', 'role': ''}],
 'publisher': "St. Martin's Press",
 'num_pages': '256',
 'publication_day': '1',
 'isbn13': '9780312853129',
 'publication_month': '9',
 'edition_information': '',
 'publication_year': '1984',
 'url': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'book_id': '5333265',
 'ratings_count': '3',
 'work_id': '5400751',
 'title': '

**List of all columns or keys**

In [5]:
print(list(line.keys()))

['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code', 'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin', 'similar_books', 'description', 'format', 'link', 'authors', 'publisher', 'num_pages', 'publication_day', 'isbn13', 'publication_month', 'edition_information', 'publication_year', 'url', 'image_url', 'book_id', 'ratings_count', 'work_id', 'title', 'title_without_series']


**Required columns or keys for analysis**
- Manually selected based on the requirement

In [6]:
cols_not_required = [""]

req_columns = [
    'isbn',
    'text_reviews_count',
    'series',
    'country_code',
    'language_code',
    'asin',
    'is_ebook',
    'average_rating',
    'kindle_asin',
    'similar_books',
    'description',
    'format',
    'link',
    'authors',
    'publisher',
    'num_pages',
    'publication_day',
    'isbn13',
    'publication_month',
    'publication_year',
    'url',
    'image_url',
    'book_id',
    'ratings_count',
    'title_without_series'
]

**Creating empty dataframe so that we can append the records dynamically**

In [7]:
# items = pd.DataFrame(columns=req_columns)
# items

,isbn,text_reviews_count,series,country_code,language_code,asin,is_ebook,average_rating,kindle_asin,similar_books,...,num_pages,publication_day,isbn13,publication_month,publication_year,url,image_url,book_id,ratings_count,title_without_series


**Parsing the json data (one at a time) and creation a dictionary while using the necessary columns**

In [2]:
def parse_fields(line):
    data = json.loads(line)
    return {
        'isbn' : data['isbn'],
        'text_reviews_count' : data['text_reviews_count'],
        'series' : data['series'],
        'country_code' : data['country_code'],
        'language_code' : data['language_code'],
        'asin' : data['asin'],
        'is_ebook' : data['is_ebook'],
        'average_rating' : data['average_rating'],
        'kindle_asin' : data['kindle_asin'],
        'similar_books' : data['similar_books'],
        'description' : data['description'],
        'format' : data['format'],
        'link' : data['link'],
        'authors' : data['authors'],
        'publisher' : data['publisher'],
        'num_pages' : data['num_pages'],
        'publication_day' : data['publication_day'],
        'isbn13' : data['isbn13'],
        'publication_month' : data['publication_month'],
        'publication_year' : data['publication_year'],
        'url' : data['url'],
        'image_url' : data['image_url'],
        'book_id' : data['book_id'],
        'ratings_count' : data['ratings_count'],
        'title_without_series' : data['title_without_series']
    }

**We will go through all the books from the dataset, chunk by chunk (10000 lines = 1 chunk) till the end and simultaneously keep adding the records into the dataframe**
- Runtime: 8-9 m
- Slightly better memory management but cost slightly extra time

In [9]:
# chunk_size = 10000

# with gzip.open("../Initial/goodreads_books.json.gz") as f:

#     json_chunk = [next(f) for _ in range(chunk_size)]
    
#     while json_chunk:
#         temp_dict_chunk = list()
#         for line in json_chunk:
#             dict_data = parse_fields(line)
#             temp_dict_chunk.append(dict_data)
        
#         temp_df = pd.DataFrame.from_dict(temp_dict_chunk)
#         items = pd.concat((items,temp_df), ignore_index=True)

#         # Read the next chunk
#         json_chunk = [next(f) for _ in range(chunk_size)]

StopIteration: 

In [11]:
# len(items)

2360000

**Insight**
- The total number of records = 2,360,655 and we have got = 2,360,000
  - We failed to read some data from the end of the file because we have chosen the chunk size as 1000 and the last remaining line count was 655.
  - So, the chunk size should be chosen carefully or the function logic should be build carefully to handle the corner cases. 

**We will use the following method to read the data from the file instead of the above one**
- Reading one line at a time
- Runtime: 4-5 m
- Slightly faster but consumes slightly more memory

In [3]:
books = []

with gzip.open("../Initial/goodreads_books.json.gz") as f:
    while True:
        # reading the line
        line = f.readline()

        # we will break the infinite loop when we reach the end of the dataset file
        if not line:
            break
        
        # parsing the line
        fields = parse_fields(line)
        books.append(fields)

**Total number of books**

In [4]:
len(books)

2360655

**Insight**
- There are 2,360,655 book records in total

In [5]:
books[0]

{'isbn': '0312853122',
 'text_reviews_count': '1',
 'series': [],
 'country_code': 'US',
 'language_code': '',
 'asin': '',
 'is_ebook': 'false',
 'average_rating': '4.00',
 'kindle_asin': '',
 'similar_books': [],
 'description': '',
 'format': 'Paperback',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'authors': [{'author_id': '604031', 'role': ''}],
 'publisher': "St. Martin's Press",
 'num_pages': '256',
 'publication_day': '1',
 'isbn13': '9780312853129',
 'publication_month': '9',
 'publication_year': '1984',
 'url': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'book_id': '5333265',
 'ratings_count': '3',
 'title_without_series': 'W.C. Fields: A Life on Film'}

**Creating a DataFrame**

In [6]:
items = pd.DataFrame.from_dict(books)

**Rows and Columns of the DataFrame**

In [7]:
print(f"Rows: {items.shape[0]}")
print(f"Columns: {items.shape[1]}")

Rows: 2360655
Columns: 25


**We don't need the `books` object anymore**
- Deleting the object from namespace and forcing Python for garbage collection (if required)
- The `del` method only removes the variable from the namespace and it does not remove the variable from the memory space
- To clear the variable from memory we can use the gc.collect() method

In [8]:
del(books)

In [15]:
gc.collect()

644

### EDA - Complete Dataset

In [10]:
items.head()

,isbn,text_reviews_count,series,country_code,language_code,asin,is_ebook,average_rating,kindle_asin,similar_books,...,num_pages,publication_day,isbn13,publication_month,publication_year,url,image_url,book_id,ratings_count,title_without_series
0,0312853122,1,[],US,,,false,4.00,,[],...,256,1,9780312853129,9,1984,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3,W.C. Fields: A Life on Film
1,0743509986,6,[],US,,,false,3.23,B000FC0PBC,"[8709549, 17074050, 28937, 158816, 228563, 112...",...,,1,9780743509985,10,2001,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,10,Good Harbor
2,,7,[189911],US,eng,B00071IKUY,false,4.03,,"[19997, 828466, 1569323, 425389, 1176674, 2627...",...,600,,,,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,"The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,0743294297,3282,[],US,eng,,false,3.49,B002ENBLOK,"[6604176, 6054190, 2285777, 82641, 7569453, 70...",...,368,14,9780743294294,7,2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,Best Friends Forever
4,0850308712,5,[],US,,,false,3.40,,[],...,,,9780850308716,,,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15,Runic Astrology: Starcraft and Timekeeping in ...


**Number of unique book_id**

In [12]:
items["book_id"].unique().shape

(2360655,)

**Insight**
- All the books have unique book_id = 2,360,655

```
Rows: 2360655
```

**Checking for null values**

In [16]:
items.isnull().sum()

isbn                    0
text_reviews_count      0
series                  0
country_code            0
language_code           0
asin                    0
is_ebook                0
average_rating          0
kindle_asin             0
similar_books           0
description             0
format                  0
link                    0
authors                 0
publisher               0
num_pages               0
publication_day         0
isbn13                  0
publication_month       0
publication_year        0
url                     0
image_url               0
book_id                 0
ratings_count           0
title_without_series    0
dtype: int64

**Insight**
- Although we can visually see empty data but null values are not there

#### `title_without_series` Column

In [63]:
items.loc[items["title_without_series"].str.len() == 0].shape

(7, 25)

In [ ]:
# items.loc[items["title_without_series"].str.len() == 0]

In [ ]:
# items.loc[items["title_without_series"].str.len() == 0, "book_id"].values

**Insight**
- There are 7 books without any title (len == 0)
  - ['2433394', '6046733', '29522179', '23280835', '10192004',
       '25423066', '7807037']

In [71]:
items.loc[items["title_without_series"].str.len() == 1].shape

(248, 25)

In [ ]:
# items.loc[items["title_without_series"].str.len() == 1]

**Insight**
- I have crosschecked some titles from the associated link and their names seems correct for most opf the cases (9 out of 10)
  - 1, NOT OKAY => Actual Title: Sunk
  - X, OKAY
  - 5, OKAY
  - Y, OKAY
  - C, OKAY
  - 6, OKAY
  - 7, OKAY
  - Q, OKAY
  - L, OKAY
  - Y, OKAY (Not as the same Y as above)
- So, we will keep the records

#### `isbn` Column Wrangling

In [18]:
items.loc[items["isbn"].str.len() < 1].shape

(983373, 25)

In [ ]:
# items.loc[items["isbn"].str.len() < 1].sample(5)

In [24]:
items.loc[(items["isbn"].str.len() > 0) & (items["isbn"].str.len() < 10)].shape

(2829, 25)

In [ ]:
# items.loc[(items["isbn"].str.len() > 0) & (items["isbn"].str.len() < 10)].sample(5)

In [34]:
items.loc[items["isbn"].str.len() > 10].shape

(0, 25)

In [35]:
items.loc[items["isbn"].str.len() == 10].shape

(1374453, 25)

In [36]:
1374453 + 2829 + 983373

2360655

In [52]:
items.loc[(items["isbn"].str.len() == 10) & (items["isbn"].str.isdigit() == False)].shape

(121243, 25)

In [ ]:
# items.loc[(items["isbn"].str.len() == 10) & (items["isbn"].str.isdigit() == False)].sample(5)

In [58]:
items.loc[(items["isbn"].str.len() == 10) & (items["isbn"].str.endswith("X") == True)].shape

(117288, 25)

In [62]:
items.loc[(items["isbn"].str.len() == 10) & (items["isbn"].str.endswith('[a-zA-Z]'))].shape

(0, 25)

**Insight**
- Any ISBN with length more than 0 less than 10 digit long or alphanumeric can be considered as invalid = 2,829
- Any ISBN with length less than 1 can be considered missing = 983,373
- There is no ISBN with length more than 10

**We don't need the `books` object anymore**
- Deleting the object from namespace and forcing Python for garbage collection (if required)
- The `del` method only removes the variable from the namespace and it does not remove the variable from the memory space
- To clear the variable from memory we can use the gc.collect() method

In [15]:
del(books)
gc.collect()

0

### Exploratory Data Analysis

**List of Columns**

In [16]:
list(items.columns)

['isbn',
 'average_rating',
 'similar_books',
 'description',
 'link',
 'authors',
 'publisher',
 'num_pages',
 'publication_day',
 'publication_month',
 'publication_year',
 'isbn13',
 'url',
 'image_url',
 'gr_book_id',
 'ratings_count',
 'title']

In [17]:
items.head()

,isbn,average_rating,similar_books,description,link,authors,publisher,num_pages,publication_day,publication_month,publication_year,isbn13,url,image_url,gr_book_id,ratings_count,title
0,,4.03,"[19997, 828466, 1569323, 425389, 1176674, 2627...",Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600,,,1987,,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,"The Unschooled Wizard (Sun Wolf and Starhawk, ..."
1,0743294297,3.49,"[6604176, 6054190, 2285777, 82641, 7569453, 70...",Addie Downs and Valerie Adler were eight when ...,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",Atria Books,368,14,7,2009,9780743294294,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,Best Friends Forever
2,1599150603,4.13,[],"Relates in vigorous prose the tale of Aeneas, ...",https://www.goodreads.com/book/show/287141.The...,"[{'author_id': '3041852', 'role': ''}]",Yesterday's Classics,162,13,9,2006,9781599150604,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46,The Aeneid for Boys and Girls
3,1934876569,4.22,"[948696, 439885, 274955, 12978730, 372986, 216...","To Kara's astonishment, she discovers that a p...",https://www.goodreads.com/book/show/6066812-al...,"[{'author_id': '19158', 'role': ''}]",Seven Seas,216,3,3,2009,9781934876565,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,6066812,98,All's Fairy in Love and War (Avalon: Web of Ma...
4,0922915113,3.81,"[287151, 1104760, 1172822, 440292, 287082, 630...","Wisdom, humor, and dark observations by the fo...",https://www.goodreads.com/book/show/287149.The...,"[{'author_id': '2983296', 'role': ''}, {'autho...",Feral House,147,1,4,2000,9780922915118,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,287149,986,The Devil's Notebook


**Checking for null values**

In [18]:
items.isnull().sum()

isbn                 0
average_rating       0
similar_books        0
description          0
link                 0
authors              0
publisher            0
num_pages            0
publication_day      0
publication_month    0
publication_year     0
isbn13               0
url                  0
image_url            0
gr_book_id           0
ratings_count        0
title                0
dtype: int64

**Insight**
- Although we can visually see empty data but null values are not there

**Looking at datatypes of the values from individual columns of a single record**
- We will extract the data from each column from any one record and check the details

In [19]:
single_record = items.iloc[0]
single_record

isbn                                                                  
average_rating                                                    4.03
similar_books        [19997, 828466, 1569323, 425389, 1176674, 2627...
description          Omnibus book club edition containing the Ladie...
link                 https://www.goodreads.com/book/show/7327624-th...
authors                           [{'author_id': '10333', 'role': ''}]
publisher                                       Nelson Doubleday, Inc.
num_pages                                                          600
publication_day                                                       
publication_month                                                     
publication_year                                                  1987
isbn13                                                                
url                  https://www.goodreads.com/book/show/7327624-th...
image_url            https://images.gr-assets.com/books/1304100136m...
gr_boo

In [20]:
for i in single_record.index:
    print(f"Column: {i} \t - Datatype: {type(single_record[i])}")

Column: isbn 	 - Datatype: <class 'str'>
Column: average_rating 	 - Datatype: <class 'str'>
Column: similar_books 	 - Datatype: <class 'list'>
Column: description 	 - Datatype: <class 'str'>
Column: link 	 - Datatype: <class 'str'>
Column: authors 	 - Datatype: <class 'list'>
Column: publisher 	 - Datatype: <class 'str'>
Column: num_pages 	 - Datatype: <class 'str'>
Column: publication_day 	 - Datatype: <class 'str'>
Column: publication_month 	 - Datatype: <class 'str'>
Column: publication_year 	 - Datatype: <class 'str'>
Column: isbn13 	 - Datatype: <class 'str'>
Column: url 	 - Datatype: <class 'str'>
Column: image_url 	 - Datatype: <class 'str'>
Column: gr_book_id 	 - Datatype: <class 'str'>
Column: ratings_count 	 - Datatype: <class 'str'>
Column: title 	 - Datatype: <class 'str'>


**Insight**
- So, the empty data are string with `length = 0`
- Datatype of `authors` and `similar_books` are of type `list`
- Rest of the data are of `string` datatype 

**Checking the datatype of the columns as a whole**

In [21]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308957 entries, 0 to 1308956
Data columns (total 17 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   isbn               1308957 non-null  object
 1   average_rating     1308957 non-null  object
 2   similar_books      1308957 non-null  object
 3   description        1308957 non-null  object
 4   link               1308957 non-null  object
 5   authors            1308957 non-null  object
 6   publisher          1308957 non-null  object
 7   num_pages          1308957 non-null  object
 8   publication_day    1308957 non-null  object
 9   publication_month  1308957 non-null  object
 10  publication_year   1308957 non-null  object
 11  isbn13             1308957 non-null  object
 12  url                1308957 non-null  object
 13  image_url          1308957 non-null  object
 14  gr_book_id         1308957 non-null  object
 15  ratings_count      1308957 non-null  object
 16  

**Insight**
- All are looking `object` because we have string values in most of the columns there apart from `authors` and `similar_books` which are of `list` type
- `average_rating` and `ratings_count` column data are of `object` datatype but they should be of numeric type for further analysis and use.
- Similarly there are other columns which we can convert into numeric
  - `["num_pages", "publication_day", "publication_month", "publication_year", "gr_book_id"]`

**Converting datatype of columns in their proper form**
- The `errors='coerce'` parameter in the `pd.to_numeric` function means that if there are any values in the specified columns that cannot be converted to numeric, those values will be set to `NaN` (Not a Number)

In [22]:
columns_to_convert = ["average_rating", "ratings_count", "num_pages", "publication_day", "publication_month", "publication_year", "gr_book_id"]

items[columns_to_convert] = items[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [23]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308957 entries, 0 to 1308956
Data columns (total 17 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   isbn               1308957 non-null  object 
 1   average_rating     1308957 non-null  float64
 2   similar_books      1308957 non-null  object 
 3   description        1308957 non-null  object 
 4   link               1308957 non-null  object 
 5   authors            1308957 non-null  object 
 6   publisher          1308957 non-null  object 
 7   num_pages          937378 non-null   float64
 8   publication_day    800719 non-null   float64
 9   publication_month  870759 non-null   float64
 10  publication_year   988556 non-null   float64
 11  isbn13             1308957 non-null  object 
 12  url                1308957 non-null  object 
 13  image_url          1308957 non-null  object 
 14  gr_book_id         1308957 non-null  int64  
 15  ratings_count      1308957 non-n

**Insight**
- We can obseve that datatypes has been converted successfully

**Checking for null values again**

In [24]:
items.isnull().sum()

isbn                      0
average_rating            0
similar_books             0
description               0
link                      0
authors                   0
publisher                 0
num_pages            371579
publication_day      508238
publication_month    438198
publication_year     320401
isbn13                    0
url                       0
image_url                 0
gr_book_id                0
ratings_count             0
title                     0
dtype: int64

**Insight**
- We can observe multiple missing values as follow

```
num_pages            371579
publication_day      508238
publication_month    438198
publication_year     320401
```
- We can simply impute them with `0`

**Imputing the missing values with `0`**

In [25]:
items.fillna(0, inplace=True)

**Checking null values after imputation**

In [26]:
items.isnull().sum()

isbn                 0
average_rating       0
similar_books        0
description          0
link                 0
authors              0
publisher            0
num_pages            0
publication_day      0
publication_month    0
publication_year     0
isbn13               0
url                  0
image_url            0
gr_book_id           0
ratings_count        0
title                0
dtype: int64

**Insight**
- We can observe that all null values has been imputed with `0`
- `0` in `["num_pages", "publication_day", "publication_month", "publication_year"]` does not make sense
- So we can consider them as missing data

**Converting ISBN and ISBN13 into numeric**

In [27]:
columns_to_convert = ["isbn", "isbn13"]

items[columns_to_convert] = items[columns_to_convert].apply(pd.to_numeric, errors='coerce')

**Checking for null values again**

In [28]:
items.isnull().sum()

isbn                 562645
average_rating            0
similar_books             0
description               0
link                      0
authors                   0
publisher                 0
num_pages                 0
publication_day           0
publication_month         0
publication_year          0
isbn13               401625
url                       0
image_url                 0
gr_book_id                0
ratings_count             0
title                     0
dtype: int64

**Insight**
- We can observe some `nan` values as these were faulty or alphanumeric ISBNs
- ISBN ideally can only be numeric and of a specific length 10 or 13.
- So, we can call them `null` values and impute them with `0` as well

**Imputing the missing values with `0`**

In [29]:
items.fillna(0, inplace=True)

**Checking null values after imputation**

In [30]:
items.isnull().sum()

isbn                 0
average_rating       0
similar_books        0
description          0
link                 0
authors              0
publisher            0
num_pages            0
publication_day      0
publication_month    0
publication_year     0
isbn13               0
url                  0
image_url            0
gr_book_id           0
ratings_count        0
title                0
dtype: int64

**Insight**
- We can observe that all null values has been imputed with `0`
- `0` in `["isbn","isbn13"]` does not make sense
- So we can consider them as missing data

In [31]:
# code stop point

# 10/0

**Checking the datatypes before export**

In [32]:
items.dtypes

isbn                 float64
average_rating       float64
similar_books         object
description           object
link                  object
authors               object
publisher             object
num_pages            float64
publication_day      float64
publication_month    float64
publication_year     float64
isbn13               float64
url                   object
image_url             object
gr_book_id             int64
ratings_count          int64
title                 object
dtype: object

**Insight**
```
isbn                 float64 - NOT OKAY
average_rating       float64 - OKAY
num_pages            float64 - NOT OKAY
publication_day      float64 - NOT OKAY
publication_month    float64 - NOT OKAY
publication_year     float64 - NOT OKAY
isbn13               float64 - NOT OKAY
gr_book_id             int64 - NOT OKAY
ratings_count          int64 - NOT OKAY
```

- We need to convert the NOT OKAY datatypes into int.

**Converting columns into integer**

In [33]:
columns_to_convert = ['isbn', 
                      'isbn13', 
                      'gr_book_id', 
                      'num_pages', 
                      'publication_day', 
                      'publication_month', 
                      'publication_year', 
                      'ratings_count']

items[columns_to_convert] = items[columns_to_convert].astype('int64')

items.dtypes

isbn                   int64
average_rating       float64
similar_books         object
description           object
link                  object
authors               object
publisher             object
num_pages              int64
publication_day        int64
publication_month      int64
publication_year       int64
isbn13                 int64
url                   object
image_url             object
gr_book_id             int64
ratings_count          int64
title                 object
dtype: object

**We will export this version of the data**
- We have proper datatypes for the columns
- We will export the data as compressed json where each record will be a single line as a json 

In [34]:
# with gzip.open('../Processed/books_ratingcount_gt15_p0.json.gz', 'wt', encoding='utf-8') as file:
#     file.write(items.to_json(orient='records', lines=True))

**Looking at missing data at different columns**

In [35]:
imputed_cols = ["average_rating", "ratings_count", "num_pages", "publication_day", "publication_month", "publication_year", "gr_book_id", "isbn", "isbn13"]

for i in imputed_cols[1:]:
    print(i," - ",len(items[items[i] == 0]))

ratings_count  -  0
num_pages  -  375062
publication_day  -  508238
publication_month  -  438198
publication_year  -  320405
gr_book_id  -  0
isbn  -  562647
isbn13  -  401626


**We will look for duplicate records**

In [36]:
columns_to_ignore = ['similar_books', 'authors']
columns_to_check = items.columns.difference(columns_to_ignore)

items.duplicated(subset=columns_to_check).sum()

0

**Insight**
- We can see that there are no duplicate records as whole
- We need to look more deeper into the duplicates columns wise

#### Deeper Look into Duplicates - Column Wise

**Duplicate `title`**

In [37]:
items["title"].duplicated().sum()

318555

**Insight**
- We have 318,555 duplicate titles

**Number of unique books among duplicates**

In [38]:
len(items.loc[items["title"].duplicated(),"title"].unique())

146168

**Insight**
- We have 146,168 duplicate titles

**We can have a look at top 25 books on the duplicate list**

In [39]:
items.loc[items["title"].duplicated(),"title"].value_counts()[:25]

title
Jane Eyre                           231
Pride and Prejudice                 222
Selected Poems                      205
Dracula                             184
Wuthering Heights                   173
Frankenstein                        170
Emma                                151
Fahrenheit 451                      143
1984                                138
The Great Gatsby                    136
The Picture of Dorian Gray          131
Anna Karenina                       125
A Christmas Carol                   117
The Hobbit                          112
Sense and Sensibility               111
Great Expectations                  111
The Secret Garden                   110
Peter Pan                           106
Madame Bovary                       102
Macbeth                             101
Hamlet                               99
Treasure Island                      95
Animal Farm                          94
Lolita                               94
Alice's Adventures in Wonderland  

**Insights**
- These are usually very famous books

**Lets deeply analyze the book - `Frankenstein`**

In [40]:
duplicate_book = items.loc[items["title"] == "Frankenstein"]
duplicate_book.shape

(171, 17)

**Insight**
- Total 171 books
- 170 duplicates in total

**We can look at the number of unique values in other columns for this specific book**

In [41]:
columns_to_ignore = ['similar_books', 'authors']
columns_to_check = items.columns.difference(columns_to_ignore)

for col in columns_to_check:
    print(f"Column: {col} \t - {len(duplicate_book[col].unique())}")

Column: average_rating 	 - 8
Column: description 	 - 110
Column: gr_book_id 	 - 171
Column: image_url 	 - 99
Column: isbn 	 - 116
Column: isbn13 	 - 147
Column: link 	 - 171
Column: num_pages 	 - 94
Column: publication_day 	 - 28
Column: publication_month 	 - 13
Column: publication_year 	 - 36
Column: publisher 	 - 111
Column: ratings_count 	 - 126
Column: title 	 - 1
Column: url 	 - 171


**Insight**
- All the books have unique `book_id`, `link` and `url`. So all books exist separately. 

In [42]:
duplicate_book["isbn"].value_counts()

isbn
0             56
866119817      1
8804405341     1
321096983      1
7350961        1
              ..
140433627      1
1631060163     1
8467039493     1
1554811031     1
192815326      1
Name: count, Length: 116, dtype: int64

In [43]:
duplicate_book["isbn13"].value_counts()

isbn13
0                25
9780141334417     1
9788420666495     1
9781493625284     1
9780866119818     1
                 ..
9780143105039     1
9780141045115     1
9781419542244     1
9780760750803     1
9780192815323     1
Name: count, Length: 147, dtype: int64

**Inshight**
- 56 books no isbn
- 25 books no isbn13

**Number of books with no isbn or no isbn13**

In [44]:
duplicate_book.loc[(duplicate_book["isbn"] == 0) & (duplicate_book["isbn13"] == 0)].shape[0]

23

**Insight**
- 23 books has no isbn or isbn13

**Number of books any one of isbn or isbn13**

In [45]:
duplicate_book.loc[(duplicate_book["isbn"] != 0) | (duplicate_book["isbn13"] != 0)].shape[0]

148

**Insight**
- 148 books are there with at least one isbn version

**Among thsese 148 books we are looking for duplicate isbn or isbn13**

In [46]:
print((duplicate_book.loc[(duplicate_book["isbn"] != 0) | (duplicate_book["isbn13"] != 0),"isbn"].value_counts()))
print((duplicate_book.loc[(duplicate_book["isbn"] != 0) | (duplicate_book["isbn13"] != 0),"isbn13"].value_counts()))

isbn
0             33
866119817      1
8804405341     1
321096983      1
7350961        1
              ..
140433627      1
1631060163     1
8467039493     1
1554811031     1
192815326      1
Name: count, Length: 116, dtype: int64
isbn13
0                2
9780141334417    1
9781493625284    1
9780451527714    1
9780195149012    1
                ..
9780143105039    1
9780141045115    1
9781419542244    1
9780760750803    1
9780192815323    1
Name: count, Length: 147, dtype: int64


**Insight**
- There is no duplicate isbn or isbn13 among these 148 books
- Only repeated isbn or isbn13 are `0` which means missing ISBNs
- Apart from that, there are 23 more books with missing ISBNs
- Finally we can state that there are no books with repeated ISBN among the books having at least one version of the ISBN

In [47]:
duplicate_book["image_url"].value_counts()[:5]

image_url
https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png    73
https://images.gr-assets.com/books/1375686080m/18489.jpg                                     1
https://images.gr-assets.com/books/1417034489m/595765.jpg                                    1
https://images.gr-assets.com/books/1427546373m/603259.jpg                                    1
https://images.gr-assets.com/books/1335360874m/11175287.jpg                                  1
Name: count, dtype: int64

**Insight**
- The only repeated `image_url` is NO PHOTO image

In [48]:
duplicate_book["description"].value_counts()[:7]

description
Mary Shelley's Frankenstein is the world's most famous Gothic novel about eccentric scientist Victor Frankenstein, who creates a grotesque creature in an unorthodox scientific experiment. Shelley's work is considered to be the world's first science fiction, with Frankenstein's monster being a symbol of science gone awry. Shelley's masterpiece has inspired numerous films, plays and other books. This, the 1831 edition, contains the author's final revisions.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

**Insight**
- `"Mary Shelley's Frankenstein is the world's most famous Gothic novel about eccentric scientist Victor Frankenstein, who creates a grotesque creature in an unorthodox scientific experiment. Shelley's work is considered to be the world's first science fiction, with Frankenstein's monster being a symbol of science gone awry. Shelley's masterpiece has inspired numerous films, plays and other books. This, the 1831 edition, contains the author's final revisions."`
  - 55 books have used the same description
- 2 books have no description
- Other descriptions are repeated as follows times 3x, 3x, 2x, 2x
  - There are no more repeated descriptions

In [49]:
duplicate_book["publisher"].value_counts()[:25]

publisher
                               27
Penguin Classics                6
Oxford University Press         5
Newton Compton                  3
Penguin                         3
Sterling                        3
Austral                         3
Dover Publications              3
Barnes  Noble Classics          3
Kaplan Publishing               2
Bantam                          2
Audible, Inc.                   2
Wordsworth Classics             2
Mondadori                       2
Edimat Libros                   2
Wordsworth Editions             2
Modern Library                  2
Bantam Books                    2
Ediciones B                     2
Barnes & Noble                  2
Gramedia Pustaka Utama          2
Signet                          2
Underwood-Miller                1
Arcturus Publishing Limited     1
Baronet Books                   1
Name: count, dtype: int64

In [50]:
duplicate_book.loc[duplicate_book["publisher"]=="Penguin Classics", ["publication_year","publication_month", "publication_day"]]

,publication_year,publication_month,publication_day
32921,2013,10,3
132147,2015,11,16
368629,2013,10,3
445869,1992,11,2
485813,2011,8,4
1172465,1986,1,7


In [51]:
duplicate_book.loc[duplicate_book["publisher"]=="Oxford University Press", ["publication_year","publication_month", "publication_day"]]

,publication_year,publication_month,publication_day
45758,2009,5,0
193530,1998,8,0
1091026,2008,10,15
1249150,1998,6,25
1288163,1969,0,0


In [52]:
duplicate_book.loc[duplicate_book["publisher"]=="Newton Compton", ["publication_year","publication_month", "publication_day"]]

,publication_year,publication_month,publication_day
38342,1994,1,1
223539,2008,2,0
1067955,2010,5,27


**Insight**
- Most of the publisher has published this book multiple times over the years

In [53]:
duplicate_book.loc[(duplicate_book["publisher"]=="Penguin Classics") & (duplicate_book["publication_year"]==2013)]

,isbn,average_rating,similar_books,description,link,authors,publisher,num_pages,publication_day,publication_month,publication_year,isbn13,url,image_url,gr_book_id,ratings_count,title
32921,141393394,3.76,"[93157, 70985, 51497, 36314, 12194, 816740, 12...",Mary Shelley's Frankenstein is the world's mos...,https://www.goodreads.com/book/show/18306732-f...,"[{'author_id': '11139', 'role': ''}]",Penguin Classics,273,3,10,2013,9780141393391,https://www.goodreads.com/book/show/18306732-f...,https://images.gr-assets.com/books/1380546334m...,18306732,454,Frankenstein
368629,143122339,3.76,"[93157, 70985, 51497, 36314, 12194, 816740, 12...",Mary Shelley's Frankenstein is the world's mos...,https://www.goodreads.com/book/show/15808305-f...,"[{'author_id': '11139', 'role': ''}, {'author_...",Penguin Classics,384,3,10,2013,9780143122333,https://www.goodreads.com/book/show/15808305-f...,https://images.gr-assets.com/books/1373997238m...,15808305,141,Frankenstein


**Insight**
- "Penguin Classics" have published the book same book same day, month and year but with different page numbers as 273 and 384
- So, two different versions
- But we can say that both are almost equally famous

In [54]:
# code stopping point

# 10/0

### Creating Modified Title to Minimize Search Space

**Removing characters apart from A-z, a-z and 0-9 and Space**

In [55]:
items["mod_title"] = items["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)

**We will make title lower case**

In [56]:
items["mod_title"] = items["mod_title"].str.lower()

**Replacing any extra spaces with a single space**

In [57]:
items["mod_title"] = items["mod_title"].str.replace("\s+", " ", regex=True)

**We will keep only those records where the `mod_title` is not empty or has a length > 0**

In [58]:
items = items[items["mod_title"].str.len() > 0]

In [59]:
len(items) 

1302659

In [60]:
items.shape

(1302659, 18)

**Insight**
- We can observe a sligt reduction in count which is expected

**Exporting the Data as Compressed JSON After Reducing Search Space**

In [61]:
# with gzip.open('../Processed/books_ratingcount_gt15_p1.json.gz', 'wt', encoding='utf-8') as file:
#     file.write(items.to_json(orient='records', lines=True))

In [ ]:
# execution stop point

# 10/0

### Removing Records With Title Containing Letters Other Than English

**Titles with letter other than Engish becomes `' '` or string of length 1 in the `mod_title` column - Removing those**

In [63]:
print(len(items[items["mod_title"].str.len() == 1]))

37931


**Getting the indics of those records**

In [64]:
items_drop = items[items["mod_title"].str.len() == 1].index
items_drop

Index([     28,      37,      38,      43,      65,      72,      85,     118,
           175,     198,
       ...
       1308680, 1308686, 1308691, 1308697, 1308789, 1308850, 1308854, 1308881,
       1308931, 1308940],
      dtype='int64', length=37931)

**We can crosscheck the net number of books that we will get after dropping the records**
- `1302659 - 37931 = 1264728`

In [65]:
print(len(items.drop(index=items_drop)))
print(1302659 - 37931)

1264728
1264728


**Dropping the records from the actual DataFrame**

In [66]:
items.drop(index=items_drop, inplace=True)

items.shape

(1264728, 18)

**Exporting the Data as Compressed JSON After Second Level of Processing**

In [69]:
# Export DataFrame to JSON with one line per record
# Compress the JSON string and write to a gzip file

with gzip.open('../Processed/books_ratingcount_gt15_p2.json.gz', 'wt', encoding='utf-8') as file:
    file.write(items.to_json(orient='records', lines=True))

In [ ]:
# execution stop point

10/0

### Testing The Exported Files

In [70]:
# testing the file  => opening gzip file streaming fashion

with gzip.open('../Processed/books_ratingcount_gt15_p2.json.gz') as file:
    for i in range(2):
        line = file.readline()

line

b'{"isbn":743294297,"average_rating":3.49,"similar_books":["6604176","6054190","2285777","82641","7569453","7010683","5941079","7052976","227709","6389685","5456247","3134684"],"description":"Addie Downs and Valerie Adler were eight when they first met and decided to be best friends forever. But, in the wake of tragedy and betrayal during their teenage years, everything changed. Val went on to fame and fortune. Addie stayed behind in their small Midwestern town. Destiny, however, had more in store for these two. And when, twenty-five years later, Val shows up at Addie\'s front door with blood on her coat and terror on her face, it is the beginning of a wild adventure for two women joined by love and history who find strength together that they could not find alone.","link":"https:\\/\\/www.goodreads.com\\/book\\/show\\/6066819-best-friends-forever","authors":[{"author_id":"9212","role":""}],"publisher":"Atria Books","num_pages":368,"publication_day":14,"publication_month":7,"publicatio

In [71]:
json.loads(line)

{'isbn': 743294297,
 'average_rating': 3.49,
 'similar_books': ['6604176',
  '6054190',
  '2285777',
  '82641',
  '7569453',
  '7010683',
  '5941079',
  '7052976',
  '227709',
  '6389685',
  '5456247',
  '3134684'],
 'description': "Addie Downs and Valerie Adler were eight when they first met and decided to be best friends forever. But, in the wake of tragedy and betrayal during their teenage years, everything changed. Val went on to fame and fortune. Addie stayed behind in their small Midwestern town. Destiny, however, had more in store for these two. And when, twenty-five years later, Val shows up at Addie's front door with blood on her coat and terror on her face, it is the beginning of a wild adventure for two women joined by love and history who find strength together that they could not find alone.",
 'link': 'https://www.goodreads.com/book/show/6066819-best-friends-forever',
 'authors': [{'author_id': '9212', 'role': ''}],
 'publisher': 'Atria Books',
 'num_pages': 368,
 'public